In [6]:
import pandas as pd
import pymongo
import datetime as dt

from anpact_scrapper import get_last_ANPACT_records

In [7]:
#Import historic data
d= pd.read_csv('01_HISTORICOS/FROST_ventas_historicas_camiones_20220304153453.csv')
d['truckTractor'] = [float(i) for i in (d['truckTractor'].str.replace(',', ''))]

In [8]:
#Set connection with MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'mexican_truckDB' database in Mongo
db = client.mexican_truckDB

In [46]:
#Insertar base de datos historica:
for i in range(0, len(d)):
    
    db.sales.insert_one({
        'date': d['Fecha'][i],
        'sales': {
            'truck4_5_ANPACT': d['truck4_5_ANPACT'][i],
            'truck6': d['truck6'][i],
            'truck7': d['truck7'][i],
            'truckTractor': d['truckTractor'][i],
            'bus5_6': d['bus5_6'][i],
            'bus7': d['bus7'][i],
            'bus8': d['bus8'][i],
            'busLongDist': d['busLongDist'][i]
        },
        'date_added' : str(dt.datetime.now()).replace('-', '').replace(' ', '').replace(':', '')[:-7]
        })    

In [24]:
#s= db.sales.find()
#for record in s:
#    print(record)

In [36]:
#Scrap latest ANPACT reports
new_record = get_last_ANPACT_records()

#Get last data point from the databse
last_record = db.sales.find_one(sort=[( '_id', pymongo.DESCENDING )])
last_record = pd.to_datetime(last_record['date'])

#Check if there is new data to be appended to database
if last_record == new_record['Fecha'][0]:
    print('Most recent ANPACT data already recorded in database! :)')
    print('----')
    
elif last_record == new_record['Fecha'][0] - pd.DateOffset(month=1):
    print('New ANPACT report published. Appending new data to database! :)')
    print('----')
    
    db.sales.insert_one({
        'date': new_record['Fecha'][0],
        'sales': {
            'truck4_5_ANPACT': new_record['truck4_5_ANPACT'][0],
            'truck6': new_record['truck6'][0],
            'truck7': new_record['truck7'][0],
            'truckTractor': new_record['truckTractor'][0],
            'bus5_6': new_record['bus5_6'][0],
            'bus7': new_record['bus7'][0],
            'bus8': new_record['bus8'][0],
            'busLongDist': new_record['busLongDist'][0]
        },
        'date_added' : str(dt.datetime.now()).replace('-', '').replace(' ', '').replace(':', '')[:-7]
        })    
    
elif last_record != new_record['Fecha'][0] - pd.DateOffset(month=1):
    print('New ANPACT report published, but last record does not match new record.')
    print('You may have skipped scrapping last month´s report... :O')
    print('----')

Most recent ANPACT data already recorded in database! :)
----
